In [1]:
import numpy as np
from NonlinearController.controllers import *
from NonlinearController.mpc_utils import *

In [2]:
A = np.array([[1,2],[3,4]])
B = np.array([[1],[2]])
Nc = 3
nx = 2
nu = 1

In [3]:
list_A = np.tile(A, (Nc, 1))
list_B = np.tile(B, (Nc, 1))

In [13]:
Phi = get_Phi(list_A, Nc, nx)

In [21]:
Gamma = get_Gamma(list_A, list_B, Nc, nx, nu)

In [22]:
x = np.ones((nx,1))
u = np.ones((nu,1))

In [23]:
Xk = np.tile(x, (Nc, 1))
Uk = np.tile(u, (Nc,1))


In [24]:
Gamma@Uk

array([[ 1.],
       [ 2.],
       [ 6.],
       [13.],
       [33.],
       [72.]])

In [39]:
#reference = np.arange(5)
reference = np.ones((2,2))

In [40]:
reference.shape[0]

2

In [34]:
r = [1,2,3]

In [41]:
len(reference)

2

In [36]:
r.shape

AttributeError: 'list' object has no attribute 'shape'